In [ ]:
# STEP 1: Install libraries
!pip install pandas folium

import pandas as pd
import folium

# STEP 2: Upload your 3 CSVs
from google.colab import files

uploaded = files.upload()  # Upload all 3: stations.csv, linyu_data.csv, incidents.csv

# STEP 3: Read CSVs
stations = pd.read_csv("stations.csv")
linyu = pd.read_csv("linyu_data.csv")
incidents = pd.read_csv("incidents.csv")

# STEP 4: Merge datasets
df = stations.merge(linyu, on="station_name", how="left")
df = df.merge(incidents, on="station_name", how="left")

# STEP 5: Create incident summary string
df['incident_summary'] = (
    "Murders: " + df['murders'].astype(str) + "<br>" +
    "Rapes: " + df['rapes'].astype(str) + "<br>" +
    "Robbery: " + df['robbery'].astype(str) + "<br>" +
    "Harassment: " + df['harassment'].astype(str)
)

# STEP 6: Create the NYC map
nyc_map = folium.Map(location=[40.75, -73.98], zoom_start=12)

for _, row in df.iterrows():
    popup = f"""
    <b>Station:</b> {row['station_name']}<br>
    <b>Train Lines:</b> {row['train_lines']}<br>
    <b>Ridership:</b> {row['ridership']}<br>
    <b>Barrier Type:</b> {row['feature_flags']}<br>
    <b>Incidents:</b><br>{row['incident_summary']}
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        tooltip=row['station_name'],
        popup=popup,
        icon=folium.Icon(color="blue")
    ).add_to(nyc_map)

# STEP 7: Save HTML
nyc_map.save("nyc_subway_map.html")

# STEP 8: Download the HTML map
files.download("nyc_subway_map.html")
